# 去中心化规划

## 目标和前提条件

准备好接受数学优化建模的挑战了吗？通过这个例子来测试您的技能,在这里您将学习如何建模和求解一个去中心化规划问题。您需要找出 - 给定一个公司的部门集合,以及这些部门可能迁移的潜在城市 - 为每个部门确定'最佳'位置,以最大化毛利润。

这个模型是 H. Paul Williams 所著《数学规划中的模型构建》第五版中的示例 10,见第 265 页和第 317-319 页。

这个建模示例属于高级水平,我们假设您已掌握 Python 和 Gurobi Python API,并且具备构建数学优化模型的高级知识。通常,这些示例的目标函数和/或约束条件比较复杂,或需要使用 Gurobi Python API 的高级功能。

**下载代码库** <br /> 
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

## 问题描述

一家大公司想要将其部分部门迁出伦敦。这样做在某些方面会降低成本(如房价更便宜、政府激励、更容易招聘等),但在其他方面会增加成本(如部门之间的通信成本)。已经计算出了每个部门在所有可能位置的成本影响。
目标是确定每个部门的位置,以最大化搬迁带来的成本降低与通信成本增加之间的总差额。

该公司包括五个部门(A、B、C、D和E)。可选择的搬迁城市有布里斯托尔和布莱顿,或者部门可以留在伦敦。这些城市(包括伦敦)中的任何一个都不能安置超过三个部门。

## 模型构建

### 集合和索引

$d,d2 \in \text{Departments}=\{A,B,C,D,E\}$

$c,c2 \in \text{Cities}=\{\text{布里斯托尔}, \text{布莱顿}, \text{伦敦}\}$

### 参数

$\text{benefit}_{d,c} \in \mathbb{R}^+$: 将部门 $d$ 搬迁到城市 $c$ 获得的收益(每年千美元)

$\text{communicationCost}_{d,c,d2,c2} \in \mathbb{R}^+$: 当部门 $d$ 搬迁到城市 $c$ 且部门 $d2$ 搬迁到城市 $c2$ 时产生的通信成本(每年千美元)

我们定义集合 $dcd2c2 = \{(d,c,d2,c2) \in \text{Departments} \times \text{Cities} \times \text{Departments} \times \text{Cities}: \text{communicationCost}_{d,c,d2,c2} > 0  \}$

### 决策变量

$\text{locate}_{d,c} \in \{0,1 \}$: 如果部门 $d$ 位于城市 $c$ 则该二元变量为1,否则为0

$y_{d,c,d2,c2} = \text{locate}_{d,c}*\text{locate}_{d2,c2} \in \{0,1 \}$: 当部门 $d$ 位于城市 $c$ 且部门 $d2$ 位于城市 $c2$ 时该辅助二元变量为1,否则为0


### 约束条件

**部门位置**: 每个部门必须且只能位于一个城市

\begin{equation}
\sum_{c \in \text{Cities}} \text{locate}_{d,c} = 1 \quad \forall d \in \text{Departments}
\end{equation}

**部门数量限制**: 任何城市容纳的部门不得超过三个

\begin{equation}
\sum_{d \in \text{Departments}} \text{locate}_{d,c} \leq 3 \quad \forall c \in \text{Cities}
\end{equation}

**逻辑约束**: 

- 如果 $y_{d,c,d2,c2} = 1$ 则 $\text{locate}_{d,c} = 1$ 且 $\text{locate}_{d2,c2} = 1$

\begin{equation}
y_{d,c,d2,c2} \leq \text{locate}_{d,c} \quad \forall (d,c,d2,c2) \in dcd2c2
\end{equation}

\begin{equation}
y_{d,c,d2,c2} \leq \text{locate}_{d2,c2} \quad \forall (d,c,d2,c2) \in dcd2c2
\end{equation}

- 如果 $\text{locate}_{d,c} = 1$ 且 $\text{locate}_{d2,c2} = 1$ 则 $y_{d,c,d2,c2} = 1$


\begin{equation}
\text{locate}_{d,c} + \text{locate}_{d2,c2} - y_{d,c,d2,c2} \leq 1 \quad  \forall (d,c,d2,c2) \in dcd2c2
\end{equation}

### 目标函数

**毛利润**: 最大化搬迁的毛利润

\begin{equation}
\text{最大化} \quad Z = \sum_{d \in \text{Departments}} \sum_{c \in \text{Cities}} \text{benefit}_{d,c}*\text{locate}_{d,c} -
\sum_{d,c,d2,c2 \in dcd2c2} \text{communicationCost}_{d,c,d2,c2}*y_{d,c,d2,c2}
\end{equation}

这个去中心化问题的线性整数规划表述实际上是此问题的二次分配表述的线性化。使用 Gurobi 9.0,您可以直接求解去中心化问题的二次分配表述,无需引入辅助变量和逻辑约束。

### 目标函数

**毛利润**: 最大化搬迁的毛利润

\begin{equation}
\text{最大化} \quad Z = \sum_{d \in \text{Departments}} \sum_{c \in \text{Cities}} \text{benefit}_{d,c}*\text{locate}_{d,c} -
\sum_{d,c,d2,c2 \in dcd2c2} \text{communicationCost}_{d,c,d2,c2}*\text{locate}_{d,c}*\text{locate}_{d2,c2}
\end{equation}

### 约束条件

**部门位置**: 每个部门必须且只能位于一个城市

\begin{equation}
\sum_{c \in \text{Cities}} \text{locate}_{d,c} = 1 \quad \forall d \in \text{Departments}
\end{equation}

**部门数量限制**: 任何城市容纳的部门不得超过三个

\begin{equation}
\sum_{d \in \text{Departments}} \text{locate}_{d,c} \leq 3 \quad \forall c \in \text{Cities}
\end{equation}

## Python 实现

我们导入 Gurobi Python 模块和其他 Python 库。

In [ ]:
%pip install gurobipy

In [1]:
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

## 输入数据

我们定义模型所需的所有输入数据。

In [ ]:
# 部门和城市列表

Deparments = ['A','B','C','D','E']
Cities = ['Bristol', 'Brighton', 'London']

# 创建字典来记录搬迁带来的收益（单位：千美元）

d2c, benefit = gp.multidict({
    ('A', 'Bristol'): 10,
    ('A', 'Brighton'): 10,
    ('A', 'London'): 0,
    ('B', 'Bristol'): 15,
    ('B', 'Brighton'): 20,
    ('B', 'London'): 0,
    ('C', 'Bristol'): 10,
    ('C', 'Brighton'): 15,
    ('C', 'London'): 0,
    ('D', 'Bristol'): 20,
    ('D', 'Brighton'): 15,
    ('D', 'London'): 0,
    ('E', 'Bristol'): 5,
    ('E', 'Brighton'): 15,
    ('E', 'London'): 0
})

# 创建字典来记录搬迁产生的通信成本（单位：千美元）

dcd2c2, communicationCost = gp.multidict({
    ('A','London','C','Bristol'): 13,
    ('A','London','C','Brighton'): 9,
    ('A','London','C','London'): 10,
    ('A','London','D','Bristol'): 19.5,
    ('A','London','D','Brighton'): 13.5,
    ('A','London','D','London'): 15,
    ('B','London','C','Bristol'): 18.2,
    ('B','London','C','Brighton'): 12.6,
    ('B','London','C','London'): 14,
    ('B','London','D','Bristol'): 15.6,
    ('B','London','D','Brighton'): 10.8,
    ('B','London','D','London'): 12,
    ('C','London','E','Bristol'): 26,
    ('C','London','E','Brighton'): 18,
    ('C','London','E','London'): 20,
    ('D','London','E','Bristol'): 9.1,
    ('D','London','E','Brighton'): 6.3,
    ('D','London','E','London'): 7,
    ('A','Bristol','C','Bristol'): 5,
    ('A','Bristol','C','Brighton'): 14,
    ('A','Bristol','C','London'): 13,
    ('A','Bristol','D','Bristol'): 7.5,
    ('A','Bristol','D','Brighton'): 21,
    ('A','Bristol','D','London'): 19.5,
    ('B','Bristol','C','Bristol'): 7,
    ('B','Bristol','C','Brighton'): 19.6,
    ('B','Bristol','C','London'): 18.2,
    ('B','Bristol','D','Bristol'): 6,
    ('B','Bristol','D','Brighton'): 16.8,
    ('B','Bristol','D','London'): 15.6,
    ('C','Bristol','E','Bristol'): 10,
    ('C','Bristol','E','Brighton'): 28,
    ('C','Bristol','E','London'): 26,
    ('D','Bristol','E','Bristol'): 3.5,
    ('D','Bristol','E','Brighton'): 9.8, 
    ('D','Bristol','E','London'): 9.1,
    ('A','Brighton','C','Bristol'): 14,
    ('A','Brighton','C','Brighton'): 5,
    ('A','Brighton','C','London'): 9,
    ('A','Brighton','D','Bristol'): 21,
    ('A','Brighton','D','Brighton'): 7.5,
    ('A','Brighton','D','London'): 13.5,
    ('B','Brighton','C','Bristol'): 19.6,
    ('B','Brighton','C','Brighton'): 7,
    ('B','Brighton','C','London'): 12.6,
    ('B','Brighton','D','Bristol'): 16.8,
    ('B','Brighton','D','Brighton'): 6,
    ('B','Brighton','D','London'): 10.8,
    ('C','Brighton','E','Bristol'): 28,
    ('C','Brighton','E','Brighton'): 10,
    ('C','Brighton','E','London'): 18,
    ('D','Brighton','E','Bristol'): 9.8,
    ('D','Brighton','E','Brighton'): 3.5,
    ('D','Brighton','E','London'): 6.3
})

## 模型部署

我们创建一个模型和变量。这些二元决策变量定义每个部门将被安置在哪个城市。

使用 Gurobi 求解二次分配问题很简单,只需将全局参数 `nonConvex` 配置为值 2。

In [ ]:
model = gp.Model('decentralization')

# 设置全局参数
model.params.nonConvex = 2

# 将部门 d 安置在城市 c
locate = model.addVars(d2c, vtype=GRB.BINARY, name="locate")

Using license file c:\gurobi\gurobi.lic
Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1


每个部门必须且只能位于一个城市。

In [ ]:
# 部门位置约束

department_location = model.addConstrs((gp.quicksum(locate[d,c] for c in Cities) == 1 for d in Deparments), 
                                    name='department_location')

任何城市容纳的部门不得超过三个。

In [ ]:
# 部门数量限制

departments_limit = model.addConstrs((gp.quicksum(locate[d,c] for d in Deparments) <= 3 for c in Cities), 
                                    name='departments_limit')

现在我们设置优化目标,即最大化毛利润。

In [6]:
model.setObjective((gp.quicksum(benefit[d,c]*locate[d,c] for d,c in d2c) 
                    - gp.quicksum(communicationCost[d,c,d2,c2]*locate[d,c]*locate[d2,c2] for d,c,d2,c2 in dcd2c2) ),
                   GRB.MAXIMIZE)

In [ ]:
# 验证模型公式

model.write('decentralizationQA.lp')

# 运行优化引擎

model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8 rows, 15 columns and 30 nonzeros
Model fingerprint: 0x2ad3c449
Model has 54 quadratic objective terms
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  QObjective range [7e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective -73.9000000
Presolve time: 0.00s
Presolved: 62 rows, 69 columns, 192 nonzeros
Variable types: 0 continuous, 69 integer (69 binary)

Root relaxation: objective -6.750000e+01, 14 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   67.50000    0   10  -73.90000   67.50000   191%     -    0s
H    0     0                   

## 分析

以下是最优搬迁方案和相关财务报告。

In [8]:
relocation_plan = pd.DataFrame(
    {key for key, var in locate.items() if var.x > 0.5},
    columns = ["Department", "City"],
)
relocation_plan.index=['']*len(relocation_plan)
relocation_plan.sort_values(["Department", "City"])

,Department,City
,A,Bristol
,D,Bristol
,B,Brighton
,C,Brighton
,E,Brighton


In [ ]:
print("\n\n_________________________________________________________________________________")
print(f"财务报告")
print("_________________________________________________________________________________")
total_benefit = 0
for c in Cities:
    for d in Deparments:
        if(locate[d,c].x > 0.5):
            total_benefit += 1000*benefit[d,c]

dollars_benefit = '${:,.2f}'.format(total_benefit)
print(f"年度总收益为 {dollars_benefit} 美元")

total_communication_cost = 0
for d,c,d2,c2 in dcd2c2:
    if(locate[d,c].x*locate[d2,c2].x > 0.5):
        total_communication_cost += 1000*communicationCost[d,c,d2,c2]

dollars_communication_cost = '${:,.2f}'.format(total_communication_cost)
print(f"年度总通信成本为 {dollars_communication_cost} 美元")

total_gross_margin = total_benefit - total_communication_cost
dollars_gross_margin = '${:,.2f}'.format(total_gross_margin)
print(f"年度总毛利润为 {dollars_gross_margin} 美元")



_________________________________________________________________________________
Financial report
_________________________________________________________________________________
The yearly total benefit is $80,000.00 dollars
The yearly total communication cost is $65,100.00 dollars
The yearly total gross margin is $14,900.00 dollars


## 参考文献

H. Paul Williams, 《数学规划中的模型构建》第五版。

版权所有 © 2020 Gurobi Optimization, LLC